# Cours TAL – Labo 3 : analyse syntaxique
Nathan Gonzalez Montes et Vincent Guidoux

## Exercice 1

### Manipulations
D'abord, on réalise les tests de performance avec le fichier obtenu `UD_French.gz` sur les fichier `fr-ud-test.conllu3` et `fr-ud-dev.conllu3` et on vérifie les résultats. Après avoir testé, on réalise notre entraînement avec le fichier `fr-ud-train.conllu3` en reprenant comme model le même fichier qu'avant, en modifiant le nom pour avoir notre propre fichier entraîné (on a mis `UD_French_train.gz` comme nom, ). Après la réalisation de notre entraînement, on relance les tests de performance avec le fichier entraîné pour voir la différence dans le résultats (améliorés grâce à notre entraînement).

### Test sans notre entraînement
#### Test file
##### Dans la ligne de commande:
`java -mx2000m -cp stanford-corenlp-3.9.2.jar edu.stanford.nlp.parser.nndep.DependencyParser -t
estFile data/fr-ud-test.conllu3 -model data/UD_French.gz`

On obtient les résultats suivants:

**OOV Words:** 608 / 10020 = 6.07%

**UAS** = 55.0699

**LAS** = 41.1577

DependencyParser parsed 10020 words in 416 sentences in 4.7s at 2129.2 w/s, 88.4 sent/s.

![image](../Screenshot/test_before_train.PNG)

#### Dev file
##### Dans la ligne de commande:
`java -mx2000m -cp stanford-corenlp-3.9.2.jar edu.stanford.nlp.parser.nndep.DependencyParser -t
estFile data/fr-ud-dev.conllu3 -model data/UD_French.gz`

On obtient les résultats suivants:

**OOV Words:** 2716 / 35771 = 7.59%

**UAS** = 57.2195

**LAS** = 43.6722

DependencyParser parsed 10020 words in 1478 sentences in 18.1s at 1972.4 w/s, 81.5 sent/s.

![image](../Screenshot/dev_before_train.PNG)

### Test avec notre entraînement
#### Entraînement
##### Dans la ligne de commande:
`java -mx2000m -cp stanford-corenlp-3.9.2.jar edu.stanford.nlp.parser.nndep.DependencyParser -trainFile data/fr-ud-train.conllu3 -model data/UD_French_train.gz -wordCutOff 3 -trainingThreads 6 -maxIter 5000`

`-wordCutOff 3` - Pour traiter seulement les mots apparaissant plus de 3 fois, ce qui évite le problème des nombres "uniques" avec un espace.

`‑trainingThreads 8` - Pour utiliser pleinement son processeur, indiquer le maximum selon le model de la documentation (On our 16-core test machines: a batch size of 10,000 runs fastest with around 6 threads; a batch size of 100,000 runs best with around 10 threads)

`‑maxIter 5000` - Pour arrêter l’entraînement après 5'000 itérations, de base il fait 20'000 itérations.

Un exemple des itérations de l'entraînement:
![image](../Screenshot/iterations.PNG)

#### Test file
##### Dans la ligne de commande:
`java -mx2000m -cp stanford-corenlp-3.9.2.jar edu.stanford.nlp.parser.nndep.DependencyParser -t
estFile data/fr-ud-test.conllu3 -model data/UD_French_train.gz`

On obtient les résultats suivants:

**OOV Words:** 1100 / 10020 = 10.98%

**UAS** = 77.8842

**LAS** = 71.4571

DependencyParser parsed 10020 words in 416 sentences in 1.4s at 7308.5 w/s, 303.4 sent/s.

![image](../Screenshot/test_after_train.PNG)

#### Dev file
##### Dans la ligne de commande:
`java -mx2000m -cp stanford-corenlp-3.9.2.jar edu.stanford.nlp.parser.nndep.DependencyParser -t
estFile data/fr-ud-dev.conllu3 -model data/UD_French_train.gz`

On obtient les résultats suivants:

**OOV Words:** 4724 / 35771 = 13.21%

**UAS** = 80.5094

**LAS** = 74.5241

DependencyParser parsed 35771 words in 1478 sentences in 2.7s at 13165.6 w/s, 544.0 sent/s.

![image](../Screenshot/dev_after_train.PNG)

### Question
Quel est le score du modèle fourni, et quel est le score du modèle que vous avez entraîné ?

Pour le modèle fourni, on obtient pour **UAS** (qui ne considère pas la relation sémantique) un score de **55.0699%** pour le fichier `test` et **57.2195%** pour le fichier `dev`. Tandis que pour **LAS** (qui considère la relation sémantique), on obtient un score de **41.1577%** pour le fichier `test` et **43.6722%** pour le fichier `dev`.

Quant au modèle que nous avons entraîné, pour **UAS** on obtient **77.8842%** pour le fichier `test` et **80.5094%** pour le fichier `dev`. Par rapport à **LAS**, on obtient un **74.5241%** comme score pour le fichier `test` et **71.4571%** pour le fichier `dev`, après avoir fait un entraînement de 5000 itérations

## Exercice 2

In [1]:
from nltk.grammar import DependencyGrammar
from nltk.corpus.reader.conll import ConllCorpusReader
from collections import Counter
from nltk.parse import (
    DependencyGraph,
    ProjectiveDependencyParser,
    NonprojectiveDependencyParser,
    )

### Vous devez lire le(s) fichier(s) UD phrase par phrase

In [2]:
filepath = 'data/fr-ud-test.conllu3'  
dependency_graphs = []

try:  
    fp = open(filepath, 'r',encoding="utf-8")
    raw_sentences = fp.read().split('\n\n')
finally:  
    fp.close()

nbr_raw_sents = len(raw_sentences) - 1 ## the file end with 4 '\n'
    
for i in range(nbr_raw_sents):
    try:
        dependency_graphs.append(DependencyGraph(raw_sentences[i], top_relation_label='root'))
    except:
        print("Error")

Error
Error


### Il faut ensuite extraire les triplets ayant une relation ‘nsubj’ (entre sujet et verbe)

In [3]:
nsubj_triples = []

for dependency_graph in dependency_graphs:
    for head, rel, dep in dependency_graph.triples():
        if rel == 'nsubj':
            nsubj_triples.append((head, dep))
            
occurences = Counter(nsubj_triples)

# sorted_by_second = sorted(list(occurences.items()), key=lambda tup: tup[1], reverse=True)

### Quels sont les 10 triplets les plus fréquents dans tout le corpus ?

In [4]:
# sorted_by_second[:10]
occurences.most_common(10)

for (triplet, nbr_occurences) in occurences.most_common(10):
    print('"{pron} {verb}" occure {nbr_occurences} fois '.format(
        pron=triplet[1][0],
        verb=triplet[0][0],
        nbr_occurences=nbr_occurences,
    ))

"Il a" occure 7 fois 
"on peut" occure 4 fois 
"il a" occure 4 fois 
"c' est" occure 3 fois 
"il contrôle" occure 3 fois 
"il faut" occure 3 fois 
"qui font" occure 2 fois 
"elle guette" occure 2 fois 
"vous avez" occure 2 fois 
"je vois" occure 2 fois 


## Exercice 3

### Démarrer le serveur

In [5]:
# !java -mx4g -cp "stanford-corenlp-3.9.2.jar;stanford-corenlp-3.9.2-models-french.jar" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000

In [9]:
from nltk.parse import CoreNLPParser

In [10]:
parser = CoreNLPParser(url='http://localhost:9000')

In [11]:
next(parser.raw_parse('J\'aime faire et rendre mes laboratoires au plus vite')).pretty_print()  

                                ROOT                                         
                                 |                                            
                                SENT                                         
                                 |                                            
                                 NP                                          
  _______________________________|___________________________________         
 |    |                PP                              |             |       
 |    |      __________|_____                          |             |        
 |    |     |                NP                        |             |       
 |    |     |     ___________|___                      |             |        
 |    |     |    |               PP                    |            AdP      
 |    |     |    |      _________|____                 |         ____|____    
 |    |     |    |     |              NP               |  

In [12]:
from nltk.corpus.reader.conll import ConllCorpusReader
from nltk.tag.perceptron import PerceptronTagger

#source : http://techstuffbrazil.blogspot.com/2017/03/quick-tutorial-to-nltk-corpus-reader-of.html

root = './data/'
test = 'fr-ud-test.conllu3'
COLUMN_TYPES = ('ignore', 
                'words', 
                'ignore', 
                'pos', 
                'ignore', 
                'ignore', 
                'tree', 
                'ignore', 
                'ignore', 
                'ignore')

testFile  = ConllCorpusReader(root=root, 
                                  fileids=test, 
                                  columntypes=COLUMN_TYPES, 
                                  encoding='utf8', 
                                  separator="\t", 
                                  tagset='universal')

test_sentences = testFile.sents()

sentences = []

for sentence in test_sentences:
    current_sentence = ''
    for word in sentence:
        current_sentence = current_sentence + word + ' '
    sentences.append(current_sentence)

TypeError: __init__() got an unexpected keyword argument 'separator'

In [9]:
trees = []
for sentence in sentences[:10]:
    try:
        trees.append(parser.raw_parse(sentence))
    except:
        print("Error")   
print('fini')

NameError: name 'sentences' is not defined

In [10]:
for tree in trees:
    for what in tree:
        print(what)